In [152]:
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score

In [153]:
df_train_labels_original = pd.read_csv('train_labels.csv',low_memory=False,index_col='building_id')
df_train_values_original = pd.read_csv('train_values.csv',low_memory=False, index_col='building_id', dtype= {
'geo_level_1_id':'uint8', 'geo_level_2_id':'uint16', 'geo_level_3_id':'uint16', 'count_floors_pre_eq':'uint8','age':'uint16', 'area_percentage':'uint16', 'height_percentage':'uint16', 
'land_surface_condition':'category', 'foundation_type':'category', 'roof_type':'category', 'ground_floor_type':'category', 'other_floor_type':'category', 'position':'category','plan_configuration':'category', 
'has_superstructure_adobe_mud':'bool', 'has_superstructure_mud_mortar_stone':'bool','has_superstructure_stone_flag':'bool', 'has_superstructure_cement_mortar_stone':'bool', 'has_superstructure_mud_mortar_brick':'bool', 'has_superstructure_cement_mortar_brick':'bool', 'has_superstructure_timber':'bool', 'has_superstructure_bamboo':'bool', 'has_superstructure_rc_non_engineered':'bool', 'has_superstructure_rc_engineered':'bool', 'has_superstructure_other':'bool', 
'legal_ownership_status':'category', 'count_families':'uint16', 
'has_secondary_use':'bool', 'has_secondary_use_agriculture':'bool', 'has_secondary_use_hotel':'bool', 'has_secondary_use_rental':'bool', 'has_secondary_use_institution':'bool', 'has_secondary_use_school':'bool', 'has_secondary_use_industry':'bool', 'has_secondary_use_health_post':'bool', 'has_secondary_use_gov_office':'bool', 'has_secondary_use_use_police':'bool', 'has_secondary_use_other':'bool',})
df_test_values_original = pd.read_csv('test_values.csv',low_memory=False, index_col='building_id', dtype= {
'geo_level_1_id':'uint8', 'geo_level_2_id':'uint16', 'geo_level_3_id':'uint16', 'count_floors_pre_eq':'uint8','age':'uint16', 'area_percentage':'uint16', 'height_percentage':'uint16', 
'land_surface_condition':'category', 'foundation_type':'category', 'roof_type':'category', 'ground_floor_type':'category', 'other_floor_type':'category', 'position':'category','plan_configuration':'category', 
'has_superstructure_adobe_mud':'bool', 'has_superstructure_mud_mortar_stone':'bool','has_superstructure_stone_flag':'bool', 'has_superstructure_cement_mortar_stone':'bool', 'has_superstructure_mud_mortar_brick':'bool', 'has_superstructure_cement_mortar_brick':'bool', 'has_superstructure_timber':'bool', 'has_superstructure_bamboo':'bool', 'has_superstructure_rc_non_engineered':'bool', 'has_superstructure_rc_engineered':'bool', 'has_superstructure_other':'bool', 
'legal_ownership_status':'category', 'count_families':'uint16', 
'has_secondary_use':'bool', 'has_secondary_use_agriculture':'bool', 'has_secondary_use_hotel':'bool', 'has_secondary_use_rental':'bool', 'has_secondary_use_institution':'bool', 'has_secondary_use_school':'bool', 'has_secondary_use_industry':'bool', 'has_secondary_use_health_post':'bool', 'has_secondary_use_gov_office':'bool', 'has_secondary_use_use_police':'bool', 'has_secondary_use_other':'bool',})

In [154]:
train_values_subset = pd.get_dummies(df_train_values_original)
train_labels_subset = df_train_labels_original['damage_grade']

validation_size = df_train_values_original.index.size - df_test_values_original.index.size

In [155]:
train_values, validation_values = (train_values_subset.iloc[0:173733], train_values_subset.iloc[173733:-1])
train_labels, validation_labels = (train_labels_subset.iloc[0:173733], train_labels_subset.iloc[173733:-1])

In [156]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')

In [157]:
redNeuronalPredictionsTrain = pd.read_csv('ensamble_train_feda.csv', index_col=0)
redNeuronalPredictionsTrain = df_train_labels_original.merge(redNeuronalPredictionsTrain, how='left', on="building_id").iloc[173733:-1].reset_index()
redNeuronalPredictionsTest = pd.read_csv('ensamble_test_feda.csv', index_col=0).reset_index()

In [158]:
xgBoostPredictionsTest = pd.read_csv('xgBoostPredictionProba.csv', index_col=0)
xgBoostPredictionsTrain = pd.read_csv('xgBoostPredictionProbaTrain.csv', index_col=0)

In [159]:
lightGBMPredictionsTest = pd.read_csv('lightGBMPredictionProba.csv', index_col=0)
lightGBMPredictionsTrain = pd.read_csv('lightGBMPredictionProbaTrain.csv', index_col=0)

In [160]:
rndForestPredictionsTest = pd.read_csv('randomForestPredictionProba', index_col=0)
rndForestPredictionsTrain = pd.read_csv('randomForestPredictionProbaTrain.csv', index_col=0)

In [161]:
logRegPredictionsTest = pd.read_csv('LogisticRegresionProbaTest.csv', index_col=0)
logRegPredictionsTrain = pd.read_csv('LogisticRegresionProbaTrain.csv', index_col=0)

In [162]:
trainData = pd.concat([rndForestPredictionsTrain, xgBoostPredictionsTrain, redNeuronalPredictionsTrain, lightGBMPredictionsTrain, logRegPredictionsTrain], axis=1 ).drop(columns=["building_id", "damage_grade"])

In [163]:
testData = pd.concat([rndForestPredictionsTest, xgBoostPredictionsTest, redNeuronalPredictionsTest, lightGBMPredictionsTest, logRegPredictionsTest], axis=1 ).drop(columns=["building_id"])

In [169]:
# esta a fuerza bruta, pero lleva poco tiempo en definitiva
scores = []
for a in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]:
    for b in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]:
        for c in [0.61, 0.62, 0.63, 0.64, 0.65, 0.56, 0.57, 0.58, 0.59]:
            for d in [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]:
                for e in [0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2]:
                    if (a+b+c+d+e != 1): continue
                    else:
                        trainData2 = trainData
                        trainData2 = trainData2.assign(dg1 = lambda x: (x.rndf1 * a) + (x.neural1 * b) + (x.xgb1 * c) + (x.logReg1 * d) + (x.gbm1 * e))
                        trainData2 = trainData2.assign(dg2 = lambda x: (x.rndf2 * a) + (x.neural2 * b) + (x.xgb2 * c) + (x.logReg2 * d) + (x.gbm2 * e))
                        trainData2 = trainData2.assign(dg3 = lambda x: (x.rndf3 * a) + (x.neural3 * b) + (x.xgb3 * c) + (x.logReg3 * d) + (x.gbm3 * e))
                        trainData2 = trainData2[['dg1', 'dg2', 'dg3']]
                        dgTest = []
                        for x in range(0, 86867):
                            dgTest.append(1 if ((trainData2.dg1[x] > trainData2.dg2[x]) and (trainData2.dg1[x] > trainData2.dg3[x])) else 2 if ((trainData2.dg2[x] > trainData2.dg3[x]) and (trainData2.dg2[x] > trainData2.dg1[x])) else 3)
                        score = f1_score(validation_labels, dgTest, average='micro')
                        scores.append([score, a, b, c, d, e])

In [171]:
max(scores, key= lambda x: x[0])

[0.7459219266234588, 0.1, 0.05, 0.65, 0.1, 0.1]

In [143]:
# esta a fuerza bruta, pero lleva poco tiempo en definitiva
scores = []
for a in [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]:
    for b in [0.05, 0.11, 0.12, 0.13, 0.14, 0.15, 0.06, 0.07, 0.08, 0.09]:
        for c in [0.9, 0.92, 0.93, 0.94, 0.85, 0.86, 0.87, 0.88, 0.89, 0.91]:
            if (a+b+c != 1): continue
            else:
                trainData2 = trainData
                trainData2 = trainData2.assign(dg1 = lambda x: (x.gbm1 * a) + (x.neural1 * b) + (x.xgb1 * c))
                trainData2 = trainData2.assign(dg2 = lambda x: (x.gbm2 * a) + (x.neural2 * b) + (x.xgb2 * c))
                trainData2 = trainData2.assign(dg3 = lambda x: (x.gbm3 * a) + (x.neural3 * b) + (x.xgb3 * c))
                trainData2 = trainData2[['dg1', 'dg2', 'dg3']]
                dgTest = []
                for x in range(0, 86867):
                    dgTest.append(1 if ((trainData2.dg1[x] > trainData2.dg2[x]) and (trainData2.dg1[x] > trainData2.dg3[x])) else 2 if ((trainData2.dg2[x] > trainData2.dg3[x]) and (trainData2.dg2[x] > trainData2.dg1[x])) else 3)
                score = f1_score(validation_labels, dgTest, average='micro')
                scores.append([score, a, b, c])

In [144]:
max(scores, key= lambda x: x[0])

[0.7464284480873059, 0.01, 0.08, 0.91]

[0.7488574487434814, 0.1, 0.3, 0.5, 0.1]
[0.7505151553524353, 0.07, 0.33, 0.53, 0.07]

In [145]:
testData = testData.assign(dg1 = lambda x: (x.gbm1 * 0.01) + (x.neural1 * 0.08) + (x.xgb1 * 0.91))
testData = testData.assign(dg2 = lambda x: (x.gbm2 * 0.01) + (x.neural2 * 0.08) + (x.xgb2 * 0.91))
testData = testData.assign(dg3 = lambda x: (x.gbm3 * 0.01) + (x.neural3 * 0.08) + (x.xgb3 * 0.91))
testData = testData[['dg1', 'dg2', 'dg3']]

In [146]:
dgFinal = []
for x in range(0, 86868):
    dgFinal.append(1 if ((testData.dg1[x] > testData.dg2[x]) and (testData.dg1[x] > testData.dg3[x])) else 2 if ((testData.dg2[x] > testData.dg3[x]) and (testData.dg2[x] > testData.dg1[x])) else 3)

In [147]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=dgFinal,
                             columns=submission_format.columns,
                             index=submission_format.index)
my_submission.to_csv('submissionEnsamble.csv')
my_submission

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3
...,...
310028,2
663567,2
1049160,2


0.74561110663428 primer intento ensamble con solver = 'lbfgs'